### Proyecto Final Grupal HENRY

**ML_1_user_rating**  
Implementación del algoritmo "Fidelity" y cálculo de rating corregido de los locales a partir de las ponderaciones de usuarios

**Autores:**

**Camino Federico**  
**Londero Walter**  
**Pizarro Hernan**  
**Urteaga Facundo**  
**Veron Cintia**   

**Resumen:** Implementación del algoritmo "Fidelity" el cual rankea la calidad de los usuarios y sus opiniones (usuarios con un mínimo de 4 reseñas realizadas). Luego, a partir de la ponderación de los usuarios, se recalcula el rating de aquellos locales que tengan reseña en su base de datos a partir de ponderar positivamente aquellas opiniones realizadas por usuarios con mejor reputación.

1.  **Carga de librerías y datos**
2.  **Generación de parámetros de usuarios**
3.  **Algoritmo de ranking de usuarios**
4.  **Cálculo de rating corregido de locales**

#### 1. Carga de librerías y datos

In [1]:
# Carga de librerías

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore') # Para evitar los molestos avisos.

In [2]:
# Carga de archivos

df_metadatos = pd.read_csv("archivos/postpro_df_metadatos.csv")
df_reviewsGoogle = pd.read_csv("archivos/postpro_df_reviewsGoogle.csv")
df_metadatos_filtered = pd.read_csv("archivos/postpro_df_metadatos_filtered.csv")
df_reviewsGoogle_filtered = pd.read_csv("archivos/postpro_df_reviewsGoogle_filtered.csv")


#### 2. Generación de parámetros de usuarios

* Cantidad de reseñas
* Diferencia de reseñas de los usuarios con el rating promedio del local
* Tipos de locales a los que el usuario realizó reseñas

In [3]:
# Merge para añadir avg_rating al df_reviewsGoogle
df_reviewsGoogle = df_reviewsGoogle.merge(df_metadatos[['gmap_id', 'avg_rating', 'general_category']], on='gmap_id', how='left')

# Renombrar la columna avg_rating a business_avg_rating
df_reviewsGoogle.rename(columns={'avg_rating': 'business_avg_rating'}, inplace=True)

# Crear la columna dif_rating
df_reviewsGoogle['dif_rating'] = df_reviewsGoogle['business_avg_rating'] - df_reviewsGoogle['rating']

# Calcular la cantidad de reseñas por usuario
user_reviews_count = df_reviewsGoogle['user_id'].value_counts().reset_index()
user_reviews_count.columns = ['user_id', 'reviews_count']

# Calcular la media de dif_rating por usuario
user_avg_dif_rating = df_reviewsGoogle.groupby('user_id')['dif_rating'].mean().reset_index()
user_avg_dif_rating.columns = ['user_id', 'avg_dif_rating']

# Calcular las reseñas por user_id y general_category
user_category_reviews = df_reviewsGoogle.groupby(['user_id', 'general_category']).size().reset_index(name='category_reviews')

# Pivote de los datos para tener general_category como columnas
user_category_pivot = user_category_reviews.pivot(index='user_id', columns='general_category', values='category_reviews').fillna(0).reset_index()

# Merge para combinar la cantidad de reseñas y la media de dif_rating por usuario
df_user_stats = user_reviews_count.merge(user_avg_dif_rating, on='user_id', how='left')

# Filtrar df_user_stats para incluir solo user_id con 8 reseñas o más
df_user_stats_filtered = df_user_stats[df_user_stats['reviews_count'] >= 4]

# Merge con df_user_stats_filtered para agregar las categorías
df_user_stats_filtered = df_user_stats_filtered.merge(user_category_pivot, on='user_id', how='left')

In [4]:
# Calcular la media de dif_rating en valor absoluto por usuario
user_avg_dif_modulo = df_reviewsGoogle.groupby('user_id')['dif_rating'].apply(lambda x: x.abs().mean()).reset_index()
user_avg_dif_modulo.columns = ['user_id', 'avg_dif_modulo']

# Merge para combinar avg_dif_modulo con df_user_stats_filtered
df_user_stats_filtered = df_user_stats_filtered.merge(user_avg_dif_modulo, on='user_id', how='left')

In [5]:
# Definir la lista de categorías
categories = ['Auto', 'Beauty', 'Buildings', 'Clothes', 'Education', 'Entertainment', 'Food', 'Health',
              'Home', 'Other', 'Religion', 'Services', 'Sports', 'Stores', 'Technology']

# Crear la columna sum_category
df_user_stats_filtered['sum_category'] = df_user_stats_filtered[categories].apply(lambda x: (x > 0).sum(), axis=1)

# Eliminar las columnas de categorías específicas
df_user_stats_filtered = df_user_stats_filtered.drop(columns=categories)

In [6]:
df_user_stats_filtered.head()

,user_id,reviews_count,avg_dif_rating,avg_dif_modulo,sum_category
0,1.077740e+20,63,-0.852381,0.852381,15
1,1.121287e+20,53,-0.813208,0.813208,15
2,1.070514e+20,49,0.336735,0.459184,12
3,1.084387e+20,43,-0.372093,0.902326,10
4,1.153560e+20,38,-0.639474,0.650000,11


In [ ]:
# reviews_count: cantidad de reseñas
# avg_dif_rating: suma de las diferencias entre la reseña del usuario y el promedio del local (podría dar 0 y no ser preciso, por ejemplo -0.5 + 0.5)
# avg_dif_modulo: suma del módulo de las diferencias entre la reseña del usuario y el promedio del local. Corrije lo anterior, dif promedio real entre reseña de usuario y valor real
# sum_Category: cantidad de categorías diferentes de locales que opinó el usuario

In [7]:
# Calcular máximo y mínimo de cada columna en df_user_stats_filtered
max_values = df_user_stats_filtered.max()
min_values = df_user_stats_filtered.min()

print("Valores máximos:")
print(max_values)
print("\nValores mínimos:")
print(min_values)

Valores máximos:
user_id           1.184396e+20
reviews_count     6.300000e+01
avg_dif_rating    3.380000e+00
avg_dif_modulo    3.380000e+00
sum_category      1.500000e+01
dtype: float64

Valores mínimos:
user_id           1.000094e+20
reviews_count     4.000000e+00
avg_dif_rating   -2.300000e+00
avg_dif_modulo    1.200000e-01
sum_category      1.000000e+00
dtype: float64


In [8]:
df_user_stats_filtered["reviews_count"].value_counts()

reviews_count
4     2880
5     1392
6      785
7      450
8      305
9      207
10     110
12      74
11      74
13      46
14      34
15      33
16      22
18      13
17      12
19      11
20       9
21       6
23       5
22       5
27       3
24       2
25       2
26       2
30       2
33       2
34       2
53       1
38       1
49       1
43       1
29       1
35       1
28       1
63       1
Name: count, dtype: int64

#### 1. Algoritmo de ranking de usuarios

In [20]:
# RANKING (De 1 a 5)

# *****5 :  +9 reseñas realizadas
#           avg_dif_Rating entre -0.5 y 0.5
#           avg_dif_modulo mayor o igual a 1
#           cantidad de categorías con al menos una reseña: 7           

# ****4 :   +9 reseñas realizadas
#           avg_dif_modulo mayor o igual a 1
#           cantidad de categorías con al menos una reseña: 7         

# ***3 :    avg_dif_modulo mayor o igual a 1
#           cantidad de categorías con al menos una reseña: 4       

# **2  :    avg_dif_modulo mayor o igual a 1

# *1   :    cualquier otro caso

In [9]:
# Definir las condiciones y las puntuaciones correspondientes
conditions = [
    (df_user_stats_filtered['reviews_count'] > 9) & 
    (df_user_stats_filtered['avg_dif_rating'].between(-0.5, 0.5, inclusive='both')) & 
    (df_user_stats_filtered['avg_dif_modulo'] <= 1) &
    (df_user_stats_filtered['sum_category'] > 6),

    (df_user_stats_filtered['reviews_count'] > 9) & 
    #(df_user_stats_filtered['avg_dif_rating'].between(-0.5, 0.5, inclusive='both')) & 
    (df_user_stats_filtered['avg_dif_modulo'] <= 1) &
    (df_user_stats_filtered['sum_category'] > 6),

    #(df_user_stats_filtered['reviews_count'] > 9) &
    (df_user_stats_filtered['sum_category'] > 3) &
    (df_user_stats_filtered['avg_dif_modulo'] <= 1),

    (df_user_stats_filtered['avg_dif_modulo'] <= 1.5),
]

choices = [5, 4, 3, 2]

# Aplicar las condiciones y asignar la columna user_rating
df_user_stats_filtered['user_rating'] = np.select(conditions, choices, default=1)

In [10]:
# Reviso cantidad por nivel, para chequear que este medianamente equilibrado

df_user_stats_filtered["user_rating"].value_counts()

user_rating
2    3871
3    1998
1     476
5      81
4      70
Name: count, dtype: int64

In [11]:
df_reviewsGoogle_filtered

,user_id,time,rating,gmap_id,date
0,1.089912e+20,1609909927056,5,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2021-01-06 05:12:07.056
1,1.112903e+20,1612849648663,5,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2021-02-09 05:47:28.663
2,1.126404e+20,1583643882296,4,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2020-03-08 05:04:42.296
3,1.174403e+20,1551938216355,5,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2019-03-07 05:56:56.355
4,1.005808e+20,1494910901933,5,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2017-05-16 05:01:41.933
...,...,...,...,...,...
52402,1.064552e+20,1563494706323,5,0x80c2b82e593589a1:0x35602b6daadce0d7,2019-07-19 00:05:06.323
52403,1.164411e+20,1560529465601,3,0x80c2b82e593589a1:0x35602b6daadce0d7,2019-06-14 16:24:25.601
52404,1.011934e+20,1571624221742,4,0x80c2b82e593589a1:0x35602b6daadce0d7,2019-10-21 02:17:01.742
52405,1.100051e+20,1555389818932,3,0x80c2b82e593589a1:0x35602b6daadce0d7,2019-04-16 04:43:38.932


#### 4. Cálculo de rating corregido de locales

In [22]:
# Se realiza un nuevo cálculo de los rating de los locales a partir de ponderar las reseñas de los usuarios con su ranking. 
# Para entender mejor, pongamos un ejemplo: Un local tiene tres reseñas: 3, 4 y 5. El rating promedio sería en este caso 4 ((3+4+5)/3).
# Supongamos que el usuario que opinó con 5 tiene 4 estrellas de rating como usuario, el rating promedio se recalcula con su peso:
# (3+4+5*4)/6 = 4.5
# En el caso de que el usuario que opinó con 3 tiene 5 estrellas de rating: (3*4+4+5)/6 = 3.5

In [12]:
# Realizar el merge para añadir user_rating a df_reviewsGoogle_filtered
df_reviewsGoogle_filtered = df_reviewsGoogle_filtered.merge(df_user_stats_filtered[['user_id', 'user_rating']], on='user_id', how='left')

# Rellenar los valores faltantes en user_rating con 1
df_reviewsGoogle_filtered['user_rating'].fillna(1, inplace=True)

# Asegurar que user_rating sea de tipo entero si es necesario
df_reviewsGoogle_filtered['user_rating'] = df_reviewsGoogle_filtered['user_rating'].astype(int)

In [13]:
df_reviewsGoogle_filtered["user_rating"].value_counts()

user_rating
1    41886
2     6502
3     3391
5      333
4      295
Name: count, dtype: int64

In [14]:
df_reviewsGoogle_filtered

,user_id,time,rating,gmap_id,date,user_rating
0,1.089912e+20,1609909927056,5,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2021-01-06 05:12:07.056,1
1,1.112903e+20,1612849648663,5,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2021-02-09 05:47:28.663,1
2,1.126404e+20,1583643882296,4,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2020-03-08 05:04:42.296,3
3,1.174403e+20,1551938216355,5,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2019-03-07 05:56:56.355,3
4,1.005808e+20,1494910901933,5,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2017-05-16 05:01:41.933,1
...,...,...,...,...,...,...
52402,1.064552e+20,1563494706323,5,0x80c2b82e593589a1:0x35602b6daadce0d7,2019-07-19 00:05:06.323,1
52403,1.164411e+20,1560529465601,3,0x80c2b82e593589a1:0x35602b6daadce0d7,2019-06-14 16:24:25.601,1
52404,1.011934e+20,1571624221742,4,0x80c2b82e593589a1:0x35602b6daadce0d7,2019-10-21 02:17:01.742,1
52405,1.100051e+20,1555389818932,3,0x80c2b82e593589a1:0x35602b6daadce0d7,2019-04-16 04:43:38.932,1


In [15]:
# Paso 2: Calcular la suma ponderada de ratings y la suma ponderada de las ponderaciones por gmap_id
df_reviewsGoogle_filtered['weighted_rating'] = df_reviewsGoogle_filtered['rating'] * df_reviewsGoogle_filtered['user_rating']
df_reviewsGoogle_filtered['weight'] = df_reviewsGoogle_filtered['user_rating']

# Agrupar por gmap_id y calcular las sumas ponderadas
grouped = df_reviewsGoogle_filtered.groupby('gmap_id').agg(
    total_weighted_rating=('weighted_rating', 'sum'),
    total_weight=('weight', 'sum')
).reset_index()

# Paso 3: Calcular avg_rating_correction dividiendo la suma ponderada de ratings entre la suma ponderada de las ponderaciones
grouped['avg_rating_correction'] = grouped['total_weighted_rating'] / grouped['total_weight']

# Paso 4: Merge de avg_rating_correction con df_metadatos_filtered
df_metadatos_filtered = df_metadatos_filtered.merge(grouped[['gmap_id', 'avg_rating_correction']], on='gmap_id', how='left')

In [16]:
df_metadatos_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2814 entries, 0 to 2813
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   address                2814 non-null   object 
 1   gmap_id                2814 non-null   object 
 2   latitude               2814 non-null   float64
 3   longitude              2814 non-null   float64
 4   category               2814 non-null   object 
 5   avg_rating             2814 non-null   float64
 6   num_of_reviews         2814 non-null   int64  
 7   Hours_day              2814 non-null   int64  
 8   Hours_night            2814 non-null   int64  
 9   general_category       2814 non-null   object 
 10  res_asian              2814 non-null   int64  
 11  res_latin              2814 non-null   int64  
 12  res_euro               2814 non-null   int64  
 13  res_fast               2814 non-null   int64  
 14  res_vegan              2814 non-null   int64  
 15  reli

In [17]:
# Contar los gmap_id únicos en cada DataFrame
gmap_ids_reviews = set(df_reviewsGoogle_filtered['gmap_id'])
gmap_ids_metadatos = set(df_metadatos_filtered['gmap_id'])

# Contar el número de gmap_id únicos en cada DataFrame
count_gmap_ids_reviews = len(gmap_ids_reviews)
count_gmap_ids_metadatos = len(gmap_ids_metadatos)

# Encontrar los gmap_id que están en ambos DataFrames
common_gmap_ids = gmap_ids_reviews.intersection(gmap_ids_metadatos)
count_common_gmap_ids = len(common_gmap_ids)

# Encontrar los gmap_id que faltan en cada DataFrame
gmap_ids_missing_in_reviews = gmap_ids_metadatos - gmap_ids_reviews
gmap_ids_missing_in_metadatos = gmap_ids_reviews - gmap_ids_metadatos

count_gmap_ids_missing_in_reviews = len(gmap_ids_missing_in_reviews)
count_gmap_ids_missing_in_metadatos = len(gmap_ids_missing_in_metadatos)

# Resultados
print(f"gmap_id únicos en df_reviewsGoogle_filtered: {count_gmap_ids_reviews}")
print(f"gmap_id únicos en df_metadatos_filtered: {count_gmap_ids_metadatos}")
print(f"gmap_id comunes en ambos DataFrames: {count_common_gmap_ids}")
print(f"gmap_id faltantes en df_reviewsGoogle_filtered: {count_gmap_ids_missing_in_reviews}")
print(f"gmap_id faltantes en df_metadatos_filtered: {count_gmap_ids_missing_in_metadatos}")

gmap_id únicos en df_reviewsGoogle_filtered: 767
gmap_id únicos en df_metadatos_filtered: 2814
gmap_id comunes en ambos DataFrames: 767
gmap_id faltantes en df_reviewsGoogle_filtered: 2047
gmap_id faltantes en df_metadatos_filtered: 0


In [18]:
df_metadatos_filtered[["avg_rating","avg_rating_correction"]].head()

,avg_rating,avg_rating_correction
0,4.4,4.565217
1,4.0,4.230769
2,4.4,NaN
3,4.3,4.777778
4,4.3,4.484848


In [19]:
# Guardar df_metadatos_filtered como un archivo CSV
#df_metadatos_filtered.to_csv('locales_LA.csv', index=False)

# Guardar df_metadatos_filtered como un archivo CSV
#df_reviewsGoogle_filtered.to_csv('reviews_LA.csv', index=False)